# Prompt Engineering Playground

Interactive notebook for testing and refining prompts with Claude using the Anthropic SDK.

## Features
- Single prompt testing with parameter controls
- Batch testing for multiple inputs
- A/B testing for prompt comparison
- Template management
- Response analysis and visualization

## 1. Setup & Configuration

Import libraries, load API key, and initialize the client.

In [ ]:
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
import pandas as pd

from prompt_playground.client import create_client, send_prompt, send_batch, count_tokens, estimate_cost
from prompt_playground.prompts import PromptTemplate, PromptLibrary, validate_prompt
from prompt_playground.analysis import calculate_metrics, compare_responses, visualize_comparison, extract_key_points, analyze_tone

load_dotenv()
console = Console()

rprint("[green]✓[/green] Libraries imported successfully")

In [ ]:
try:
    client = create_client()
    rprint("[green]✓[/green] Anthropic client initialized")
except ValueError as e:
    rprint(f"[red]✗[/red] {e}")
    rprint("[yellow]![/yellow] Please set ANTHROPIC_API_KEY in your .env file")

In [ ]:
model_selector = widgets.Dropdown(
    options=[
        'claude-sonnet-4-5-20250929',
        'claude-opus-4-5-20251101',
        'claude-3-7-sonnet-20250219',
        'claude-3-5-haiku-20241022',
    ],
    value='claude-sonnet-4-5-20250929',
    description='Model:',
    style={'description_width': '100px'}
)

temperature_slider = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.1,
    description='Temperature:',
    style={'description_width': '100px'}
)

max_tokens_slider = widgets.IntSlider(
    value=4096,
    min=256,
    max=8192,
    step=256,
    description='Max Tokens:',
    style={'description_width': '100px'}
)

display(widgets.VBox([
    widgets.HTML("<h3>Model Configuration</h3>"),
    model_selector,
    temperature_slider,
    max_tokens_slider
]))

session_responses = []

## 2. Single Prompt Testing

Test individual prompts with full parameter control.

In [ ]:
prompt_input = widgets.Textarea(
    value='',
    placeholder='Enter your prompt here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='120px'),
    style={'description_width': '100px'}
)

system_input = widgets.Textarea(
    value='',
    placeholder='Optional system prompt...',
    description='System:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '100px'}
)

execute_button = widgets.Button(
    description='Execute Prompt',
    button_style='primary',
    icon='play'
)

output_area = widgets.Output()

def on_execute_click(b):
    with output_area:
        output_area.clear_output()
        
        prompt = prompt_input.value
        system = system_input.value if system_input.value else None
        
        if not prompt:
            rprint("[red]Please enter a prompt[/red]")
            return
        
        validation = validate_prompt(prompt)
        if not validation['valid']:
            rprint(f"[yellow]Warning:[/yellow] {', '.join(validation['issues'])}")
        
        rprint("[blue]Sending prompt...[/blue]")
        
        try:
            response = send_prompt(
                prompt=prompt,
                model=model_selector.value,
                system=system,
                temperature=temperature_slider.value,
                max_tokens=max_tokens_slider.value,
                client=client
            )
            
            session_responses.append(response)
            
            rprint(Panel(response['text'], title="[bold green]Response[/bold green]", expand=False))
            
            metrics = calculate_metrics(response)
            
            table = Table(title="Response Metrics")
            table.add_column("Metric", style="cyan")
            table.add_column("Value", style="green")
            
            table.add_row("Model", response['model'])
            table.add_row("Words", str(metrics['word_count']))
            table.add_row("Characters", str(metrics['char_count']))
            table.add_row("Input Tokens", str(metrics['input_tokens']))
            table.add_row("Output Tokens", str(metrics['output_tokens']))
            table.add_row("Total Tokens", str(metrics['total_tokens']))
            table.add_row("Estimated Cost", f"${metrics['estimated_cost']:.6f}")
            table.add_row("Sentences", str(metrics['sentence_count']))
            table.add_row("Paragraphs", str(metrics['paragraph_count']))
            
            console.print(table)
            
        except Exception as e:
            rprint(f"[red]Error:[/red] {str(e)}")

execute_button.on_click(on_execute_click)

display(widgets.VBox([
    widgets.HTML("<h3>Single Prompt Testing</h3>"),
    prompt_input,
    system_input,
    execute_button,
    output_area
]))

## 3. Batch Testing

Test multiple inputs with the same prompt template.

In [ ]:
batch_prompts_input = widgets.Textarea(
    value='',
    placeholder='Enter prompts, one per line...',
    description='Prompts:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': '100px'}
)

batch_execute_button = widgets.Button(
    description='Execute Batch',
    button_style='success',
    icon='list'
)

batch_output = widgets.Output()

def on_batch_execute(b):
    with batch_output:
        batch_output.clear_output()
        
        prompts_text = batch_prompts_input.value
        if not prompts_text:
            rprint("[red]Please enter at least one prompt[/red]")
            return
        
        prompts = [p.strip() for p in prompts_text.split('\n') if p.strip()]
        
        rprint(f"[blue]Executing {len(prompts)} prompts...[/blue]")
        
        try:
            responses = send_batch(
                prompts=prompts,
                model=model_selector.value,
                temperature=temperature_slider.value,
                max_tokens=max_tokens_slider.value,
                client=client
            )
            
            session_responses.extend(responses)
            
            df = compare_responses(responses)
            
            display(HTML(df[['response_id', 'word_count', 'output_tokens', 'estimated_cost']].to_html()))
            
            rprint("\n[bold]Responses:[/bold]")
            for idx, response in enumerate(responses):
                rprint(Panel(
                    response['text'][:200] + ('...' if len(response['text']) > 200 else ''),
                    title=f"[bold]Response {idx + 1}[/bold]",
                    expand=False
                ))
            
            total_cost = sum(r['estimated_cost'] for r in df.to_dict('records'))
            rprint(f"\n[green]Total Cost:[/green] ${total_cost:.6f}")
            
        except Exception as e:
            rprint(f"[red]Error:[/red] {str(e)}")

batch_execute_button.on_click(on_batch_execute)

display(widgets.VBox([
    widgets.HTML("<h3>Batch Testing</h3>"),
    batch_prompts_input,
    batch_execute_button,
    batch_output
]))

## 4. A/B Testing

Compare multiple prompt variants with the same input.

In [ ]:
ab_input = widgets.Textarea(
    value='',
    placeholder='Enter the common input/question...',
    description='Input:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '100px'}
)

variant_a = widgets.Textarea(
    value='',
    placeholder='Prompt variant A (use {input} for substitution)...',
    description='Variant A:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '100px'}
)

variant_b = widgets.Textarea(
    value='',
    placeholder='Prompt variant B (use {input} for substitution)...',
    description='Variant B:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '100px'}
)

ab_execute_button = widgets.Button(
    description='Run A/B Test',
    button_style='warning',
    icon='columns'
)

ab_output = widgets.Output()

def on_ab_execute(b):
    with ab_output:
        ab_output.clear_output()
        
        input_text = ab_input.value
        variants = [variant_a.value, variant_b.value]
        variants = [v for v in variants if v.strip()]
        
        if not input_text:
            rprint("[red]Please enter input text[/red]")
            return
        
        if len(variants) < 2:
            rprint("[red]Please enter at least 2 variants[/red]")
            return
        
        prompts = [v.format(input=input_text) for v in variants]
        
        rprint(f"[blue]Testing {len(prompts)} variants...[/blue]")
        
        try:
            responses = send_batch(
                prompts=prompts,
                model=model_selector.value,
                temperature=temperature_slider.value,
                max_tokens=max_tokens_slider.value,
                client=client
            )
            
            session_responses.extend(responses)
            
            df = compare_responses(responses)
            display(HTML(df.to_html()))
            
            rprint("\n[bold]Side-by-Side Comparison:[/bold]\n")
            
            for idx, response in enumerate(responses):
                variant_letter = chr(65 + idx)
                rprint(Panel(
                    response['text'],
                    title=f"[bold cyan]Variant {variant_letter}[/bold cyan]",
                    expand=False
                ))
                
                tone = analyze_tone(response['text'])
                rprint(f"  Tone: {tone['formality']}, Complexity: {tone['complexity']}, Perspective: {tone['perspective']}\n")
            
            import matplotlib.pyplot as plt
            fig = visualize_comparison(responses, metric='tokens')
            plt.show()
            
        except Exception as e:
            rprint(f"[red]Error:[/red] {str(e)}")

ab_execute_button.on_click(on_ab_execute)

display(widgets.VBox([
    widgets.HTML("<h3>A/B Testing</h3>"),
    ab_input,
    variant_a,
    variant_b,
    ab_execute_button,
    ab_output
]))

## 5. Prompt Templates

Manage and use prompt templates with variable substitution.

In [ ]:
library = PromptLibrary('prompt_templates.json')

template_name_input = widgets.Text(
    value='',
    placeholder='Template name...',
    description='Name:',
    style={'description_width': '100px'}
)

template_text_input = widgets.Textarea(
    value='',
    placeholder='Template with {variables} in curly braces...',
    description='Template:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': '100px'}
)

template_vars_input = widgets.Text(
    value='',
    placeholder='variable1, variable2, ...',
    description='Variables:',
    style={'description_width': '100px'}
)

save_template_button = widgets.Button(
    description='Save Template',
    button_style='info',
    icon='save'
)

list_templates_button = widgets.Button(
    description='List Templates',
    button_style='info',
    icon='list'
)

template_output = widgets.Output()

def on_save_template(b):
    with template_output:
        template_output.clear_output()
        
        name = template_name_input.value
        text = template_text_input.value
        vars_text = template_vars_input.value
        
        if not name or not text or not vars_text:
            rprint("[red]Please fill all fields[/red]")
            return
        
        variables = [v.strip() for v in vars_text.split(',')]
        
        template = PromptTemplate(text, variables)
        
        if not template.validate():
            rprint("[yellow]Warning:[/yellow] Template variables don't match the template text")
        
        library.save(name, template)
        rprint(f"[green]✓[/green] Template '{name}' saved successfully")

def on_list_templates(b):
    with template_output:
        template_output.clear_output()
        
        templates = library.list_all()
        
        if not templates:
            rprint("[yellow]No templates saved yet[/yellow]")
            return
        
        table = Table(title="Saved Templates")
        table.add_column("Name", style="cyan")
        table.add_column("Variables", style="green")
        
        for name in templates:
            template = library.load(name)
            table.add_row(name, ', '.join(template.variables))
        
        console.print(table)

save_template_button.on_click(on_save_template)
list_templates_button.on_click(on_list_templates)

display(widgets.VBox([
    widgets.HTML("<h3>Prompt Templates</h3>"),
    template_name_input,
    template_text_input,
    template_vars_input,
    widgets.HBox([save_template_button, list_templates_button]),
    template_output
]))

In [ ]:
load_template_dropdown = widgets.Dropdown(
    options=library.list_all(),
    description='Load:',
    style={'description_width': '100px'}
)

refresh_dropdown_button = widgets.Button(
    description='Refresh List',
    button_style='',
    icon='refresh'
)

fill_template_button = widgets.Button(
    description='Load & Fill',
    button_style='primary',
    icon='edit'
)

template_fill_output = widgets.Output()

def on_refresh_dropdown(b):
    load_template_dropdown.options = library.list_all()

def on_fill_template(b):
    with template_fill_output:
        template_fill_output.clear_output()
        
        if not load_template_dropdown.value:
            rprint("[red]Please select a template[/red]")
            return
        
        template = library.load(load_template_dropdown.value)
        
        rprint(f"[cyan]Template:[/cyan] {template.template}")
        rprint(f"[cyan]Variables:[/cyan] {', '.join(template.variables)}\n")
        
        var_inputs = {}
        for var in template.variables:
            var_inputs[var] = widgets.Text(
                description=f"{var}:",
                style={'description_width': '100px'}
            )
        
        test_button = widgets.Button(
            description='Test Template',
            button_style='success'
        )
        
        result_output = widgets.Output()
        
        def on_test_template(b):
            with result_output:
                result_output.clear_output()
                
                values = {var: widget.value for var, widget in var_inputs.items()}
                
                try:
                    filled = template.fill(**values)
                    rprint(Panel(filled, title="[bold]Filled Prompt[/bold]", expand=False))
                    
                    response = send_prompt(
                        prompt=filled,
                        model=model_selector.value,
                        temperature=temperature_slider.value,
                        max_tokens=max_tokens_slider.value,
                        client=client
                    )
                    
                    session_responses.append(response)
                    
                    rprint(Panel(response['text'], title="[bold green]Response[/bold green]", expand=False))
                    
                    metrics = calculate_metrics(response)
                    rprint(f"\n[cyan]Tokens:[/cyan] {metrics['total_tokens']} | [cyan]Cost:[/cyan] ${metrics['estimated_cost']:.6f}")
                    
                except ValueError as e:
                    rprint(f"[red]Error:[/red] {str(e)}")
                except Exception as e:
                    rprint(f"[red]API Error:[/red] {str(e)}")
        
        test_button.on_click(on_test_template)
        
        display(widgets.VBox([
            widgets.HTML("<h4>Fill Template Variables</h4>"),
            *var_inputs.values(),
            test_button,
            result_output
        ]))

refresh_dropdown_button.on_click(on_refresh_dropdown)
fill_template_button.on_click(on_fill_template)

display(widgets.VBox([
    widgets.HTML("<h3>Use Template</h3>"),
    widgets.HBox([load_template_dropdown, refresh_dropdown_button]),
    fill_template_button,
    template_fill_output
]))

## 6. Results Analysis

Analyze and visualize session results.

In [ ]:
analyze_button = widgets.Button(
    description='Analyze Session',
    button_style='success',
    icon='chart-bar'
)

export_button = widgets.Button(
    description='Export to CSV',
    button_style='info',
    icon='download'
)

clear_button = widgets.Button(
    description='Clear Session',
    button_style='danger',
    icon='trash'
)

analysis_output = widgets.Output()

def on_analyze(b):
    with analysis_output:
        analysis_output.clear_output()
        
        if not session_responses:
            rprint("[yellow]No responses in session yet[/yellow]")
            return
        
        rprint(f"[bold]Session Statistics[/bold] ({len(session_responses)} responses)\n")
        
        df = compare_responses(session_responses)
        
        total_tokens = df['total_tokens'].sum()
        total_cost = df['estimated_cost'].sum()
        avg_words = df['word_count'].mean()
        
        table = Table(title="Session Summary")
        table.add_column("Metric", style="cyan")
        table.add_column("Value", style="green")
        
        table.add_row("Total Responses", str(len(session_responses)))
        table.add_row("Total Tokens", str(total_tokens))
        table.add_row("Total Cost", f"${total_cost:.6f}")
        table.add_row("Avg Words/Response", f"{avg_words:.1f}")
        table.add_row("Avg Cost/Response", f"${total_cost / len(session_responses):.6f}")
        
        console.print(table)
        
        rprint("\n[bold]Detailed Metrics:[/bold]")
        display(HTML(df.to_html()))
        
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        axes[0, 0].bar(df['response_id'], df['word_count'], color='skyblue')
        axes[0, 0].set_title('Word Count by Response')
        axes[0, 0].set_xlabel('Response')
        axes[0, 0].set_ylabel('Words')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        axes[0, 1].bar(df['response_id'], df['total_tokens'], color='lightcoral')
        axes[0, 1].set_title('Token Usage by Response')
        axes[0, 1].set_xlabel('Response')
        axes[0, 1].set_ylabel('Tokens')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        axes[1, 0].bar(df['response_id'], df['estimated_cost'], color='lightgreen')
        axes[1, 0].set_title('Cost by Response')
        axes[1, 0].set_xlabel('Response')
        axes[1, 0].set_ylabel('Cost (USD)')
        axes[1, 0].tick_params(axis='x', rotation=45)
        
        cumulative_cost = df['estimated_cost'].cumsum()
        axes[1, 1].plot(range(1, len(cumulative_cost) + 1), cumulative_cost, marker='o', color='purple')
        axes[1, 1].set_title('Cumulative Cost Over Session')
        axes[1, 1].set_xlabel('Response Number')
        axes[1, 1].set_ylabel('Cumulative Cost (USD)')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

def on_export(b):
    with analysis_output:
        analysis_output.clear_output(wait=True)
        
        if not session_responses:
            rprint("[yellow]No responses to export[/yellow]")
            return
        
        df = compare_responses(session_responses)
        
        filename = f"session_results_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filename, index=False)
        
        rprint(f"[green]✓[/green] Results exported to {filename}")

def on_clear(b):
    global session_responses
    session_responses = []
    with analysis_output:
        analysis_output.clear_output()
        rprint("[green]✓[/green] Session cleared")

analyze_button.on_click(on_analyze)
export_button.on_click(on_export)
clear_button.on_click(on_clear)

display(widgets.VBox([
    widgets.HTML("<h3>Results Analysis</h3>"),
    widgets.HBox([analyze_button, export_button, clear_button]),
    analysis_output
]))

## Session Info

Quick view of current session status.

In [ ]:
rprint(f"[bold cyan]Current Configuration:[/bold cyan]")
rprint(f"Model: {model_selector.value}")
rprint(f"Temperature: {temperature_slider.value}")
rprint(f"Max Tokens: {max_tokens_slider.value}")
rprint(f"\n[bold cyan]Session Status:[/bold cyan]")
rprint(f"Responses: {len(session_responses)}")
if session_responses:
    total_cost = sum(calculate_metrics(r)['estimated_cost'] for r in session_responses)
    rprint(f"Total Cost: ${total_cost:.6f}")